# Dataset description

## Wyscout Players DB

Wyscout is a platform that allows any interested person to pull data related to football players from almost every league in the world. The data was extracted from the public repository of griffsben -> https://github.com/griffisben

The data contains information such as Player name, Team, Competition, Position, and other basic data, also in-game stats amd other season stats.

This dataset was chosen due to personal interest in football data analytics.

The dataset chosen was

# Data extraction - Cloning Repository (fastest way) 

Repo was deleted after extraction

In [1]:
import os
import shutil
import glob
import subprocess

# Folder to store the cloned repository
repo_folder = "./Wyscout_Prospect_Research/"
output_folder = "../data/football-players-DB/"

# Make sure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Clone the repository (only if it doesn't exist already)
if not os.path.exists(repo_folder):
    print("Cloning the repository...")
    subprocess.run(["git", "clone", "https://github.com/griffisben/Wyscout_Prospect_Research.git", repo_folder])

# Path to the folder where CSVs are located
csv_folder_path = os.path.join(repo_folder, "Main App")

# Find all CSV files in the directory
csv_files = glob.glob(f"{csv_folder_path}/*.csv")

# Copy CSV files to the output folder
for file_path in csv_files:
    file_name = os.path.basename(file_path)
    destination_path = os.path.join(output_folder, file_name)
    shutil.copy(file_path, destination_path)
    print(f"Copied {file_name} to {destination_path}")

print("All CSV files have been copied to the destination folder.")

Cloning the repository...
Copied 1. HNL 20-21.csv to ../data/football-players-DB/1. HNL 20-21.csv
Copied 1. HNL 21-22.csv to ../data/football-players-DB/1. HNL 21-22.csv
Copied 1. HNL 22-23.csv to ../data/football-players-DB/1. HNL 22-23.csv
Copied 1. HNL 23-24.csv to ../data/football-players-DB/1. HNL 23-24.csv
Copied 1. HNL 24-25.csv to ../data/football-players-DB/1. HNL 24-25.csv
Copied 2. Bundesliga 20-21.csv to ../data/football-players-DB/2. Bundesliga 20-21.csv
Copied 2. Bundesliga 21-22.csv to ../data/football-players-DB/2. Bundesliga 21-22.csv
Copied 2. Bundesliga 22-23.csv to ../data/football-players-DB/2. Bundesliga 22-23.csv
Copied 2. Bundesliga 23-24.csv to ../data/football-players-DB/2. Bundesliga 23-24.csv
Copied 2. Bundesliga 24-25.csv to ../data/football-players-DB/2. Bundesliga 24-25.csv
Copied 2. HNL 20-21.csv to ../data/football-players-DB/2. HNL 20-21.csv
Copied 2. HNL 21-22.csv to ../data/football-players-DB/2. HNL 21-22.csv
Copied 2. HNL 22-23.csv to ../data/footb

# EDA w/ PySpark

The data will be concatenated into one single df with PySpark. 888 files from the leagues and seasons, and aprox 280 MB.
As for now, not all files have the same schema, hence only the files with the same schema are going to be used. Future work includes handling mismatching schema to fit all files in the same dataset!

The data is separated by season, so additional data processing is performed.

In [3]:
# Calculating the total size of the CSV files in the folder
import os

# Path to the folder where the CSV files are stored
folder_path = "../data/football-players-DB/"

# Initialize variables to store the total size in bytes, KB, and MB
total_size_bytes = 0
total_size_kb = 0
total_size_mb = 0

# Iterate over each file in the directory
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    # Check if the file is a regular file
    if os.path.isfile(file_path):
        # Get the size of the file in bytes
        file_size_bytes = os.path.getsize(file_path)
        
        # Add the file size to the total size
        total_size_bytes += file_size_bytes

# Calculate the total size in KB and MB
total_size_kb = total_size_bytes / 1024
total_size_mb = total_size_kb / 1024

# Print the total size in KB and MB
print(f"Total size: {total_size_kb:.2f} KB")
print(f"Total size: {total_size_mb:.2f} MB")

Total size: 291390.93 KB
Total size: 284.56 MB


In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

# Create a Spark session
spark = SparkSession.builder \
        .appName("FootballData") \
        .master("local[*]") \ 
        .getOrCreate()

# asignar todos los cores disponibles con * en local[*]

# print(spark)

In [23]:
import os

# Path to the folder where the CSVs are stored
folder_path = "../data/football-players-DB/"

# List of all CSV files
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".csv")]

# Initialize an empty DataFrame to store the final concatenated result
final_df = None
schema = None

# Iterate over each CSV file
for i, file in enumerate(csv_files):
    file_name = os.path.basename(file)  # Get the file name without the path
    print(f"Processing file {i+1}/{len(csv_files)}: {file_name}")
    
    # Extract the season from the last 4 or 5 characters of the file name (without the '.csv')
    season = file_name[-9:-4] if "-" in file_name[-9:-4] else file_name[-8:-4]
    print(f"Detected season: {season}")
    
    # Read each CSV file with schema inference
    df = spark.read.csv(file, header=True, inferSchema=True)
    
    # Add a new column 'season' with the extracted season value using lit()
    df = df.withColumn("season", lit(season))
    
    # If it's the first file, use its schema as the base schema
    if schema is None:
        schema = df.schema
        final_df = df
    else:
        # Check if the current file has the same schema as the base schema
        if df.schema == schema:
            # Concatenate the DataFrame if the schema matches
            final_df = final_df.unionByName(df)
        else:
            print(f"Schema mismatch found in file: {file_name}")
            # Optionally handle the mismatch by adjusting the schema

Processing file 1/888: 1. HNL 20-21.csv
Detected season: 20-21
Processing file 2/888: 1. HNL 21-22.csv
Detected season: 21-22
Schema mismatch found in file: 1. HNL 21-22.csv
Processing file 3/888: 1. HNL 22-23.csv
Detected season: 22-23
Processing file 4/888: 1. HNL 23-24.csv
Detected season: 23-24
Schema mismatch found in file: 1. HNL 23-24.csv
Processing file 5/888: 1. HNL 24-25.csv
Detected season: 24-25
Schema mismatch found in file: 1. HNL 24-25.csv
Processing file 6/888: 2. Bundesliga 20-21.csv
Detected season: 20-21
Processing file 7/888: 2. Bundesliga 21-22.csv
Detected season: 21-22
Processing file 8/888: 2. Bundesliga 22-23.csv
Detected season: 22-23
Schema mismatch found in file: 2. Bundesliga 22-23.csv
Processing file 9/888: 2. Bundesliga 23-24.csv
Detected season: 23-24
Schema mismatch found in file: 2. Bundesliga 23-24.csv
Processing file 10/888: 2. Bundesliga 24-25.csv
Detected season: 24-25
Schema mismatch found in file: 2. Bundesliga 24-25.csv
Processing file 11/888: 2

In [5]:
# Show the final schema of the concatenated DataFrame
final_df.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Full name: string (nullable = true)
 |-- Wyscout id: integer (nullable = true)
 |-- Team: string (nullable = true)
 |-- Team within selected timeframe: string (nullable = true)
 |-- Team logo: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Primary position: string (nullable = true)
 |-- Primary position, %: integer (nullable = true)
 |-- Secondary position: string (nullable = true)
 |-- Secondary position, %: integer (nullable = true)
 |-- Third position: string (nullable = true)
 |-- Third position, %: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Birthday: date (nullable = true)
 |-- Market value: integer (nullable = true)
 |-- Contract expires: date (nullable = true)
 |-- Matches played: integer (nullable = true)
 |-- Minutes played: integer (nullable = true)
 |-- Goals: integer (nullable = true)
 |-- xG: double (nullable = true)
 |-- Assists: intege

In [5]:
# Show some rows to ensure everything is concatenated properly
final_df.show(5)

+------------+-----------------+----------+-------------+------------------------------+--------------------+-----------+--------+----------------+-------------------+------------------+---------------------+--------------+-----------------+---+----------+------------+----------------+--------------+--------------+-----+----+-------+---+------------+------------+-------------+----------------+-----+------+------+-------+-----------------------------------+----------------------+----------------------+---------------------+-------------------+----------------------+--------------------+--------------------+--------------------+------------------+------------+------------+-------------------+---------+----------------+-----------------------------------+------------+-----------------+------------------------+---------+----------+-----------------+-----+------------+------------------+------------------+--------------+--------------+-------------------+------------------------------+-----

In [6]:
# Show summary statistics of the DataFrame
final_df.describe().show() 

+-------+------------+---------------+------------------+-------------------+------------------------------+--------------------+-----------+------------+----------------+-------------------+------------------+---------------------+--------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------+--------------------+-------+------------------+------------------+-----------------------------------+----------------------+----------------------+---------------------+-------------------+----------------------+--------------------+--------------------+--------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-----------------------------------+-------------------+------------------+------------------------+-------------------+------------------+-

In [6]:
# count the number of unique Competition in the dataset with names
final_df.select("Competition").distinct().count()

384

In [8]:
# what objet is final_df
type(final_df)

pyspark.sql.dataframe.DataFrame

In [29]:
spark.stop()

# RDDs introduction

In [26]:
# Convert final_df to an RDD
final_rdd = final_df.rdd

# Show the first 5 rows of the RDD
final_rdd.take(5)

[Row(Player='I. Ivušić', Full name='Ivica Ivušić', Wyscout id=215498, Team='Pafos', Team within selected timeframe='Osijek', Team logo='https://cdn5.wyscout.com/photos/team/public/g34232_120x120.png', Competition='1. Division', Position='GK', Primary position='GK', Primary position, %=100, Secondary position=None, Secondary position, %=0, Third position=None, Third position, %=0, Age=26, Birthday=datetime.date(1995, 2, 1), Market value=2500000, Contract expires=datetime.date(2026, 5, 31), Matches played=36, Minutes played=3493, Goals=0, xG=0.0, Assists=0, xA=0.0, Duels per 90=0.56, Duels won, %=80.95, Birth country='Croatia', Passport country="['Croatia']", Foot='right', Height=195, Weight=78, On loan=False, Successful defensive actions per 90=1.72, Defensive duels per 90=0.05, Defensive duels won, %=50.0, Aerial duels per 9035=0.37, Aerial duels won, %=78.57, Sliding tackles per 90=0.0, PAdj Sliding tackles=0.0, Shots blocked per 90=0.0, Interceptions per 90=1.69, PAdj Interceptions=2

In [6]:
# Count players by competition - All seasons (may be repeated players)b
competition_count = final_rdd.map(lambda row: (row.Competition, 1))\
                             .reduceByKey(lambda a, b: a + b)\
                             .collect()

for competition, count in competition_count:
    print(f"Competition: {competition}, Number of Players: {count}")


Competition: Süper Lig, Number of Players: 208
Competition: None, Number of Players: 16449
Competition: Prva Liga, Number of Players: 216
Competition: U15 Liga, Number of Players: 1
Competition: 1. Lig, Number of Players: 87
Competition: Change Liga, Number of Players: 2
Competition: Amapaense, Number of Players: 2
Competition: Liga MX, Number of Players: 1257
Competition: Piauiense, Number of Players: 7
Competition: Primera División, Number of Players: 1574
Competition: Calcutta Premier Division A, Number of Players: 1
Competition: MLSZ U17 Országos Bajnokság, Number of Players: 1
Competition: Primera Division RFEF, Number of Players: 427
Competition: Ligue 2, Number of Players: 43
Competition: Challenger Pro League, Number of Players: 189
Competition: Primera Division, Number of Players: 25
Competition: U21 League, Number of Players: 8
Competition: 3. Division, Number of Players: 12
Competition: 2. Bundesliga, Number of Players: 334
Competition: National 1, Number of Players: 18
Comp

In [7]:
# Calculate average age of players
ages = final_rdd.map(lambda x: x.Age)
total_age = ages.reduce(lambda a, b: a + b)
count = ages.count()
average_age = total_age / count

print(f"Average Age: {average_age:.2f} years")

Average Age: 25.24 years


In [27]:
final_rdd.count()

77988

# Spark-it or Pandas-it?

In [10]:
# Conver the final_df to a Pandas DataFrame
final_df_pandas = final_df.toPandas()

# Display the first 5 rows of the Pandas DataFrame
final_df_pandas.head()

,Player,Full name,Wyscout id,Team,Team within selected timeframe,Team logo,Competition,Position,Primary position,"Primary position, %",...,Back passes received as GK per 90,Exits per 90,Aerial duels per 90119,Free kicks per 90,Direct free kicks per 90,"Direct free kicks on target, %",Corners per 90,Penalties taken,"Penalty conversion, %",season
0,I. Ivušić,Ivica Ivušić,215498,Pafos,Osijek,https://cdn5.wyscout.com/photos/team/public/g3...,1. Division,GK,GK,100,...,0.00,1.35,0.37,NaN,NaN,NaN,0.00,0,0.0,20-21
1,I. Filipović,Ivan Filipović,264159,Dinamo Zagreb,Slaven Belupo,https://cdn5.wyscout.com/photos/team/public/23...,1. HNL,GK,GK,100,...,0.00,1.80,0.49,NaN,NaN,NaN,0.00,0,0.0,20-21
2,I. Nevistić,Ivan Nevistić,318346,Dinamo Zagreb,Rijeka,https://cdn5.wyscout.com/photos/team/public/23...,1. HNL,GK,GK,100,...,0.00,0.98,0.31,NaN,NaN,NaN,0.00,0,0.0,20-21
3,D. Livaković,Dominik Livaković,215476,Fenerbahçe,Dinamo Zagreb,https://cdn5.wyscout.com/photos/team/public/47...,Süper Lig,GK,GK,100,...,0.00,0.79,0.20,NaN,NaN,NaN,0.00,0,0.0,20-21
4,K. Lulić,Karlo Lulić,273293,Bari,Slaven Belupo,https://cdn5.wyscout.com/photos/team/public/19...,Serie B,RCMF3,RCMF3,73,...,0.15,NaN,2.35,0.39,0.03,0.0,0.69,0,0.0,20-21


In [11]:
# Count players by competition - All seasons (may be repeated players)b
competition_count = final_df_pandas["Competition"].value_counts()

for competition, count in competition_count.items():
    print(f"Competition: {competition}, Number of Players: {count}")

Competition: Serie A, Number of Players: 2791
Competition: Serie B, Number of Players: 2656
Competition: Serie C, Number of Players: 1757
Competition: J1 League, Number of Players: 1742
Competition: Primera División, Number of Players: 1574
Competition: Liga de Expansión MX, Number of Players: 1322
Competition: J2 League, Number of Players: 1290
Competition: Premier League, Number of Players: 1287
Competition: Liga MX, Number of Players: 1257
Competition: First League, Number of Players: 1052
Competition: Eliteserien, Number of Players: 1046
Competition: Ekstraklasa, Number of Players: 999
Competition: K League 2, Number of Players: 904
Competition: MLS, Number of Players: 900
Competition: I Liga, Number of Players: 878
Competition: K League 1, Number of Players: 872
Competition: Veikkausliiga, Number of Players: 804
Competition: Arabian Gulf League, Number of Players: 767
Competition: A.LeCoq Premium Liiga, Number of Players: 728
Competition: Liga Pro, Number of Players: 724
Competiti

In [12]:
# Calculate average age of players
average_age = final_df_pandas["Age"].mean()

print(f"Average Age: {average_age:.2f} years")

Average Age: 25.24 years


# API requests from football-data.org: Comparission between use of RDD and Pandas in data fetching

call of games of all leagues available in the endpoint of the API, shut it down after 5 minutes, then perform stats from the data using both RDDs and Pandas

https://www.football-data.org/

In [31]:
import requests
import pandas as pd
import time
from pyspark.sql import SparkSession

# API setup
API_KEY = 'XXXX' 
API_URL = 'https://api.football-data.org/v4/matches'
HEADERS = {
    'X-Auth-Token': API_KEY
}

# Function to fetch match data from the API with rate limiting handling
def fetch_all_matches():
    response = requests.get(API_URL, headers=HEADERS)
    if response.status_code == 200:
        return response.json().get('matches', [])
    elif response.status_code == 429:  # Rate limit reached
        retry_after = int(response.headers.get("Retry-After", 60))  # Default retry after 60 seconds
        print(f"Rate limit hit. Retrying after {retry_after} seconds.")
        time.sleep(retry_after)
        return []
    else:
        print(f"Error {response.status_code}: {response.text}")
        return []

# Step 2: Fetch matches for 5 minutes with rate limiting
start_time = time.time()
matches = []

while time.time() - start_time < 300:  # Limit to 5 minutes
    matches_batch = fetch_all_matches()
    if matches_batch:
        matches.extend(matches_batch)
        print(f"Fetched {len(matches_batch)} matches. Total matches: {len(matches)}")
    else:
        print("No more data or rate limit hit.")
        break

    time.sleep(5)  # Adding delay between requests to avoid hitting rate limit

Fetched 4 matches. Total matches: 4
Fetched 4 matches. Total matches: 8
Fetched 4 matches. Total matches: 12
Fetched 4 matches. Total matches: 16
Fetched 4 matches. Total matches: 20
Fetched 4 matches. Total matches: 24
Fetched 4 matches. Total matches: 28
Fetched 4 matches. Total matches: 32
Fetched 4 matches. Total matches: 36
Fetched 4 matches. Total matches: 40
Fetched 4 matches. Total matches: 44
Fetched 4 matches. Total matches: 48
Fetched 4 matches. Total matches: 52
Fetched 4 matches. Total matches: 56
Fetched 4 matches. Total matches: 60
Fetched 4 matches. Total matches: 64
Fetched 4 matches. Total matches: 68
Fetched 4 matches. Total matches: 72
Fetched 4 matches. Total matches: 76
Fetched 4 matches. Total matches: 80
Fetched 4 matches. Total matches: 84
Fetched 4 matches. Total matches: 88
Fetched 4 matches. Total matches: 92
Fetched 4 matches. Total matches: 96
Fetched 4 matches. Total matches: 100
Fetched 4 matches. Total matches: 104
Fetched 4 matches. Total matches: 108


Convert matches to Pandas

In [53]:
# Show first 5 rows of the Pandas dataframe
matches_df = pd.DataFrame(matches)
matches_df.head()

,area,competition,season,id,utcDate,status,matchday,stage,group,lastUpdated,homeTeam,awayTeam,score,odds,referees
0,"{'id': 2032, 'name': 'Brazil', 'code': 'BRA', ...","{'id': 2013, 'name': 'Campeonato Brasileiro Sé...","{'id': 2257, 'startDate': '2024-04-13', 'endDa...",494048,2024-10-17T00:45:00Z,FINISHED,30,REGULAR_SEASON,None,2024-10-17T02:58:03Z,"{'id': 3984, 'name': 'Fortaleza EC', 'shortNam...","{'id': 1766, 'name': 'CA Mineiro', 'shortName'...","{'winner': 'DRAW', 'duration': 'REGULAR', 'ful...",{'msg': 'Activate Odds-Package in User-Panel t...,[]
1,"{'id': 2032, 'name': 'Brazil', 'code': 'BRA', ...","{'id': 2013, 'name': 'Campeonato Brasileiro Sé...","{'id': 2257, 'startDate': '2024-04-13', 'endDa...",494051,2024-10-17T00:45:00Z,FINISHED,30,REGULAR_SEASON,None,2024-10-17T02:43:24Z,"{'id': 1776, 'name': 'São Paulo FC', 'shortNam...","{'id': 1780, 'name': 'CR Vasco da Gama', 'shor...","{'winner': 'HOME_TEAM', 'duration': 'REGULAR',...",{'msg': 'Activate Odds-Package in User-Panel t...,[]
2,"{'id': 2032, 'name': 'Brazil', 'code': 'BRA', ...","{'id': 2013, 'name': 'Campeonato Brasileiro Sé...","{'id': 2257, 'startDate': '2024-04-13', 'endDa...",494045,2024-10-17T23:00:00Z,TIMED,30,REGULAR_SEASON,None,2024-10-17T00:20:54Z,"{'id': 1779, 'name': 'SC Corinthians Paulista'...","{'id': 1768, 'name': 'CA Paranaense', 'shortNa...","{'winner': None, 'duration': 'REGULAR', 'fullT...",{'msg': 'Activate Odds-Package in User-Panel t...,[]
3,"{'id': 2032, 'name': 'Brazil', 'code': 'BRA', ...","{'id': 2013, 'name': 'Campeonato Brasileiro Sé...","{'id': 2257, 'startDate': '2024-04-13', 'endDa...",494047,2024-10-17T23:00:00Z,TIMED,30,REGULAR_SEASON,None,2024-10-17T00:20:54Z,"{'id': 1783, 'name': 'CR Flamengo', 'shortName...","{'id': 1765, 'name': 'Fluminense FC', 'shortNa...","{'winner': None, 'duration': 'REGULAR', 'fullT...",{'msg': 'Activate Odds-Package in User-Panel t...,[]
4,"{'id': 2032, 'name': 'Brazil', 'code': 'BRA', ...","{'id': 2013, 'name': 'Campeonato Brasileiro Sé...","{'id': 2257, 'startDate': '2024-04-13', 'endDa...",494051,2024-10-17T00:45:00Z,FINISHED,30,REGULAR_SEASON,None,2024-10-17T02:43:24Z,"{'id': 1776, 'name': 'São Paulo FC', 'shortNam...","{'id': 1780, 'name': 'CR Vasco da Gama', 'shor...","{'winner': 'HOME_TEAM', 'duration': 'REGULAR',...",{'msg': 'Activate Odds-Package in User-Panel t...,[]


Convert matches to RDD

In [43]:
# Show the first 5 rows of the DataFrame
spark = SparkSession.builder \
    .appName("FootballDataFetch") \
    .getOrCreate()

sc = spark.sparkContext

matches_rdd = sc.parallelize(matches)
matches_rdd.take(5)

[{'area': {'id': 2032,
   'name': 'Brazil',
   'code': 'BRA',
   'flag': 'https://crests.football-data.org/764.svg'},
  'competition': {'id': 2013,
   'name': 'Campeonato Brasileiro Série A',
   'code': 'BSA',
   'type': 'LEAGUE',
   'emblem': 'https://crests.football-data.org/bsa.png'},
  'season': {'id': 2257,
   'startDate': '2024-04-13',
   'endDate': '2024-12-08',
   'currentMatchday': 30,
   'winner': None},
  'id': 494048,
  'utcDate': '2024-10-17T00:45:00Z',
  'status': 'FINISHED',
  'matchday': 30,
  'stage': 'REGULAR_SEASON',
  'group': None,
  'lastUpdated': '2024-10-17T02:58:03Z',
  'homeTeam': {'id': 3984,
   'name': 'Fortaleza EC',
   'shortName': 'Fortaleza',
   'tla': 'FEC',
   'crest': 'https://crests.football-data.org/3984.png'},
  'awayTeam': {'id': 1766,
   'name': 'CA Mineiro',
   'shortName': 'Mineiro',
   'tla': 'CAM',
   'crest': 'https://crests.football-data.org/1766.png'},
  'score': {'winner': 'DRAW',
   'duration': 'REGULAR',
   'fullTime': {'home': 1, 'away

In [35]:
# print schema of matches_rdd
matches_rdd.take(1)

[{'area': {'id': 2032,
   'name': 'Brazil',
   'code': 'BRA',
   'flag': 'https://crests.football-data.org/764.svg'},
  'competition': {'id': 2013,
   'name': 'Campeonato Brasileiro Série A',
   'code': 'BSA',
   'type': 'LEAGUE',
   'emblem': 'https://crests.football-data.org/bsa.png'},
  'season': {'id': 2257,
   'startDate': '2024-04-13',
   'endDate': '2024-12-08',
   'currentMatchday': 30,
   'winner': None},
  'id': 494048,
  'utcDate': '2024-10-17T00:45:00Z',
  'status': 'FINISHED',
  'matchday': 30,
  'stage': 'REGULAR_SEASON',
  'group': None,
  'lastUpdated': '2024-10-17T02:58:03Z',
  'homeTeam': {'id': 3984,
   'name': 'Fortaleza EC',
   'shortName': 'Fortaleza',
   'tla': 'FEC',
   'crest': 'https://crests.football-data.org/3984.png'},
  'awayTeam': {'id': 1766,
   'name': 'CA Mineiro',
   'shortName': 'Mineiro',
   'tla': 'CAM',
   'crest': 'https://crests.football-data.org/1766.png'},
  'score': {'winner': 'DRAW',
   'duration': 'REGULAR',
   'fullTime': {'home': 1, 'away

Complete Schema is:

[{'area': {'id': 2032,
   'name': 'Brazil',
   'code': 'BRA',
   'flag': 'https://crests.football-data.org/764.svg'},
  'competition': {'id': 2013,
   'name': 'Campeonato Brasileiro Série A',
   'code': 'BSA',
   'type': 'LEAGUE',
   'emblem': 'https://crests.football-data.org/bsa.png'},
  'season': {'id': 2257,
   'startDate': '2024-04-13',
   'endDate': '2024-12-08',
   'currentMatchday': 30,
   'winner': None},
  'id': 494048,
  'utcDate': '2024-10-17T00:45:00Z',
  'status': 'FINISHED',
  'matchday': 30,
  'stage': 'REGULAR_SEASON',
  'group': None,
  'lastUpdated': '2024-10-17T02:58:03Z',
  'homeTeam': {'id': 3984,
   'name': 'Fortaleza EC',
   'shortName': 'Fortaleza',
   'tla': 'FEC',
   'crest': 'https://crests.football-data.org/3984.png'},
  'awayTeam': {'id': 1766,
   'name': 'CA Mineiro',
   'shortName': 'Mineiro',
   'tla': 'CAM',
   'crest': 'https://crests.football-data.org/1766.png'},
  'score': {'winner': 'DRAW',
   'duration': 'REGULAR',
   'fullTime': {'home': 1, 'away': 1},
   'halfTime': {'home': 1, 'away': 0}},
  'odds': {'msg': 'Activate Odds-Package in User-Panel to retrieve odds.'},
  'referees': []}]

Games by competition RDDs vs Pandas

In [39]:
# how many games by competition are in the dataset - RDDs

competition_rdd = matches_rdd.map(lambda match: (match['competition']['name'], 1))

# Reduce by key to count games per competition
competition_counts_rdd = competition_rdd.reduceByKey(lambda a, b: a + b)

# Collect the result and print
competition_counts = competition_counts_rdd.collect()
for competition, count in competition_counts:
    print(f"Competition: {competition}, Games: {count}")

Competition: Campeonato Brasileiro Série A, Games: 152


In [41]:
# how many games by competition in the dataset - Pandas

# Create a new column 'competition_name' from the nested 'name' in 'competition'
matches_df['competition_name'] = matches_df['competition'].apply(lambda x: x['name'])

# Now, group by the new 'competition_name' column and count the entries
competition_counts_df = matches_df.groupby('competition_name').size().reset_index(name='Counts')

# Display the results
print(competition_counts_df)

                competition_name  Counts
0  Campeonato Brasileiro Série A     152


Avg goals and diff of goals by Game vs competition: RDDs vs Pandas

In [48]:
# For each observation in the RDD, create a new column or key, that has the sum of the goals scored by the home and away teams minus the average of the goals scored by the home and away teams in that competition.
# RDDs

# Function to calculate total goals per match
def total_goals(match):
    # Safely get home and away goals, defaulting to 0 if they are None
    home_goals = match['score']['fullTime']['home'] if match['score']['fullTime']['home'] is not None else 0
    away_goals = match['score']['fullTime']['away'] if match['score']['fullTime']['away'] is not None else 0
    return home_goals + away_goals

# RDD operation to calculate total goals and competition name
total_goals_rdd = matches_rdd.map(lambda match: (match['competition']['name'], total_goals(match)))

# Calculate competition averages
# This produces (competition_name, (total_goals, 1)) tuples
comp_totals = total_goals_rdd.map(lambda x: (x[0], (x[1], 1)))
comp_totals = comp_totals.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
comp_averages = comp_totals.map(lambda x: (x[0], x[1][0] / x[1][1]))

# Join averages back to the original matches
# Map matches to include competition average
matches_with_avg = matches_rdd.map(lambda match: (match['competition']['name'], (match, total_goals(match))))
matches_with_avg = matches_with_avg.join(comp_averages)

# Calculate the difference from the average
matches_diff = matches_with_avg.map(lambda x: {
    'match': x[1][0][0],
    'total_goals': x[1][0][1],
    'comp_average': x[1][1],
    'diff_from_avg': x[1][0][1] - x[1][1]
})

# See the results
print(matches_diff.take(1))

[{'match': {'area': {'id': 2032, 'name': 'Brazil', 'code': 'BRA', 'flag': 'https://crests.football-data.org/764.svg'}, 'competition': {'id': 2013, 'name': 'Campeonato Brasileiro Série A', 'code': 'BSA', 'type': 'LEAGUE', 'emblem': 'https://crests.football-data.org/bsa.png'}, 'season': {'id': 2257, 'startDate': '2024-04-13', 'endDate': '2024-12-08', 'currentMatchday': 30, 'winner': None}, 'id': 494048, 'utcDate': '2024-10-17T00:45:00Z', 'status': 'FINISHED', 'matchday': 30, 'stage': 'REGULAR_SEASON', 'group': None, 'lastUpdated': '2024-10-17T02:58:03Z', 'homeTeam': {'id': 3984, 'name': 'Fortaleza EC', 'shortName': 'Fortaleza', 'tla': 'FEC', 'crest': 'https://crests.football-data.org/3984.png'}, 'awayTeam': {'id': 1766, 'name': 'CA Mineiro', 'shortName': 'Mineiro', 'tla': 'CAM', 'crest': 'https://crests.football-data.org/1766.png'}, 'score': {'winner': 'DRAW', 'duration': 'REGULAR', 'fullTime': {'home': 1, 'away': 1}, 'halfTime': {'home': 1, 'away': 0}}, 'odds': {'msg': 'Activate Odds-Pa

In [54]:
# For each observation in the RDD, create a new column or key, that has the sum of the goals scored by the home and away teams minus the average of the goals scored by the home and away teams in that competition.
# Pandas

# Ensure the score values are integers and handle None by converting them to zero
matches_df['total_goals'] = matches_df.apply(
    lambda row: (0 if row['score']['fullTime']['home'] is None else row['score']['fullTime']['home']) +
                (0 if row['score']['fullTime']['away'] is None else row['score']['fullTime']['away']),
    axis=1
)

# Calculate the average goals per competition
competition_avg = matches_df.groupby(matches_df['competition'].apply(lambda x: x['name']))['total_goals'].mean().reset_index()
competition_avg.columns = ['competition_name', 'avg_goals']

# Merge the average back to the original DataFrame
matches_df['competition_name'] = matches_df['competition'].apply(lambda x: x['name'])
matches_df = matches_df.merge(competition_avg, on='competition_name')

# Calculate the difference from the average
matches_df['diff_from_avg'] = matches_df['total_goals'] - matches_df['avg_goals']

# Display the result
print(matches_df[['competition_name', 'total_goals', 'avg_goals', 'diff_from_avg']])

                  competition_name  total_goals  avg_goals  diff_from_avg
0    Campeonato Brasileiro Série A            2       1.25           0.75
1    Campeonato Brasileiro Série A            3       1.25           1.75
2    Campeonato Brasileiro Série A            0       1.25          -1.25
3    Campeonato Brasileiro Série A            0       1.25          -1.25
4    Campeonato Brasileiro Série A            3       1.25           1.75
..                             ...          ...        ...            ...
147  Campeonato Brasileiro Série A            0       1.25          -1.25
148  Campeonato Brasileiro Série A            3       1.25           1.75
149  Campeonato Brasileiro Série A            2       1.25           0.75
150  Campeonato Brasileiro Série A            0       1.25          -1.25
151  Campeonato Brasileiro Série A            0       1.25          -1.25

[152 rows x 4 columns]


Stop Sparl

In [55]:
spark.stop()